In [26]:
import torch
from transformers import PhiForCausalLM, AutoModelForCausalLM, AutoTokenizer, AdamW
from peft import LoftQConfig, LoraConfig, get_peft_model
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm

In [19]:
import os
try:
    os.chdir("Y:\Python\script\Study\Dacon_0201")
except:
    pass

os.getcwd()

'Y:\\Python\\script\\Study\\Dacon_0201'

In [20]:
device = "cuda"

In [7]:
torch.set_default_device("cuda")

# Load model and tokenizer
model_origin = AutoModelForCausalLM.from_pretrained("daekeun-ml/phi-2-ko-v0.1", torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained("daekeun-ml/phi-2-ko-v0.1", trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [27]:
# 데이터 로드
data = pd.read_csv('./data/train.csv')

# 데이터 포맷팅 및 토크나이징
formatted_data = []
for _, row in tqdm(data.iterrows()):
    for q_col in ['질문_1', '질문_2']:
        for a_col in ['답변_1', '답변_2', '답변_3', '답변_4', '답변_5']:
            # 질문과 답변 쌍을 </s> token으로 연결
            input_text = row[q_col] + tokenizer.eos_token + row[a_col]
            input_ids = tokenizer.encode(input_text, return_tensors='pt')
            formatted_data.append(input_ids)
print('Done.')

TypeError: _refine_defaults_read() got multiple values for argument 'defaults'

In [10]:
CFG = {
    'LR' : 3e-5, # Learning Rate
    'EPOCHS' : 10, # 학습 Epoch
}

In [11]:
?LoraConfig

Init signature:
LoraConfig(
    peft_type: Union[str, peft.utils.peft_types.PeftType, NoneType] = None,
    auto_mapping: Optional[dict] = None,
    base_model_name_or_path: Optional[str] = None,
    revision: Optional[str] = None,
    task_type: Union[str, peft.utils.peft_types.TaskType, NoneType] = None,
    inference_mode: bool = False,
    r: 'int' = 8,
    target_modules: 'Optional[Union[List[str], str]]' = None,
    lora_alpha: 'int' = 8,
    lora_dropout: 'float' = 0.0,
    fan_in_fan_out: 'bool' = False,
    bias: 'str' = 'none',
    modules_to_save: 'Optional[List[str]]' = None,
    init_lora_weights: "bool | Literal['gaussian', 'loftq']" = True,
    layers_to_transform: 'Optional[Union[List[int], int]]' = None,
    layers_pattern: 'Optional[Union[List[str], str]]' = None,
    rank_pattern: 'Optional[dict]' = <factory>,
    alpha_pattern: 'Optional[dict]' = <factory>,
    megatron_config: 'Optional[dict]' = None,
    megatron_core: 'Optional[str]' = 'megatron.core',
    loftq_

In [14]:
loftq_config = LoftQConfig(loftq_bits = 4)
lora_config = LoraConfig(r = 4, lora_alpha = 4, lora_dropout = 0.05, init_lora_weights = 'loftq', loftq_config = loftq_config)

In [28]:
model = get_peft_model(model_origin, lora_config)


===================================BUG REPORT===================================
The following directories listed in your path were found to be non-existent: {WindowsPath('C'), WindowsPath('/Users/yjg10/anaconda3/lib')}
The following directories listed in your path were found to be non-existent: {WindowsPath('/matplotlib_inline.backend_inline'), WindowsPath('module')}
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
The following directories listed in your path were found to be non-existent: {WindowsPath('/usr/local/cuda/lib64')}
DEBUG: Possible options found for libcudart.so: set()
CUDA SETUP: PyTorch settings found: CUDA_VERSION=118, Highest Compute Capability: 8.9.
CUDA SETUP: To manually override the PyTorch CUDA version please see:https://github.com/TimDettmers/bitsandbytes/blob/main/how_to_use_nonpytorch_cuda.md
CUDA SETUP: Loading binary C:\Users\yjg10\anaconda3\lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.so...
argum

C:\Users\yjg10\anaconda3\lib\site-packages\bitsandbytes\cuda_setup\main.py:166: UserWarning: Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes


  warn(msg)
C:\Users\yjg10\anaconda3\lib\site-packages\bitsandbytes\cuda_setup\main.py:166: UserWarning: C:\Users\yjg10\anaconda3 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)


RuntimeError: 
        CUDA Setup failed despite GPU being available. Please run the following command to get more information:

        python -m bitsandbytes

        Inspect the output of the command and see if you can locate CUDA libraries. You might need to add them
        to your LD_LIBRARY_PATH. If you suspect a bug, please take the information from python -m bitsandbytes
        and open an issue at: https://github.com/TimDettmers/bitsandbytes/issues

In [6]:
optimizer = AdamW(model.parameters(), lr=CFG['LR'])
model.train()

C:\Users\yjg10\anaconda3\lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(66676, 2560)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (final_layernorm): LayerNorm((2560,),

In [7]:
# 모델 학습
for epoch in range(CFG['EPOCHS']):
    total_loss = 0
    progress_bar = tqdm(enumerate(formatted_data), total=len(formatted_data))
    for batch_idx, batch in progress_bar:
        # 데이터를 GPU단으로 이동
        batch = batch.to(device)
        outputs = model(batch, labels=batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()

        # 진행률 표시줄에 평균 손실 업데이트
        progress_bar.set_description(f"Epoch {epoch+1} - Avg Loss: {total_loss / (batch_idx+1):.4f}")

    # 에폭의 평균 손실을 출력
    print(f"Epoch {epoch+1}/{CFG['EPOCHS']}, Average Loss: {total_loss / len(formatted_data)}")

# 모델 저장
model.save_pretrained("./hansoldeco-kogpt2_3")
tokenizer.save_pretrained("./hansoldeco-kogpt2_3")

Epoch 1 - Avg Loss: 2.1156:   1%|▎                                                 | 42/6440 [01:02<2:39:47,  1.50s/it]


KeyboardInterrupt: 

In [ ]:

# Korean 
inputs = tokenizer("머신러닝은 ", return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_length=200)
text = tokenizer.batch_decode(outputs)[0]
print(text)

# English 
inputs = tokenizer('''def print_prime(n):
   """
   Print all primes between 1 and n
   """''', return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_length=200)
text = tokenizer.batch_decode(outputs)[0]
print(text)